In [16]:
# Familiar imports
import numpy as np
import pandas as pd

# For ordinal encoding categorical variables, splitting data
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

# For training random forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import mutual_info_regression
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, roc_auc_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
import datetime
import os

In [17]:
# Load the training data
rawtrain = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv", index_col=0)
rawtest = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv", index_col=0)

In [18]:
train = rawtrain.copy()
y = train['target']
features = train.drop(['target'], axis=1)

In [19]:
X = features.copy()
X.shape

(600000, 100)

In [20]:
test = rawtest.copy()
test.shape

(540000, 100)

In [21]:
from tensorflow import keras
from tensorflow.keras import layers, callbacks, regularizers
from keras import backend as K
import tensorflow as tf

In [22]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14967522451212243301
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9898950656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12047642508982174312
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:2a:00.0, compute capability: 8.6"
]


In [26]:
from keras import metrics
METRICS = [
    metrics.RootMeanSquaredError(name='rms'),
    metrics.MeanAbsoluteError(name='mae')
]
ENCODING_DIM = 16 #Desired Dimension
BATCH_SIZE = 64
EPOCHS = 100

In [36]:
#input_shape = [X.shape[1]]
len_input_output = X_train.shape[-1]

def make_dae(metrics=METRICS):
    dae = keras.Sequential([
        layers.Dense(units=len_input_output*2, 
              activation="relu", input_shape=(len_input_output,)),
        layers.Dropout(0.5), #Add dropout layers if required 
        layers.Dense(units=len_input_output*2, activation="relu"),
        layers.Dense(units=len_input_output*2, activation="relu"),
        layers.Dense(units=len_input_output, activation="linear"),
    ])
    dae.compile(
        optimizer='adam', 
        loss='mean_squared_error',
        metrics=[metrics]
    )
    return dae

In [37]:
from numba import cuda

In [38]:
X_train = X
noise = np.random.normal(loc=0, scale=0.5, size=X_train.shape)
X_train_noisy = X_train + noise

In [39]:
dae = make_dae()
history = dae.fit(
    X_train_noisy,
    X_train,
    batch_size = BATCH_SIZE,
    epochs = EPOCHS
    )

Epoch 1/100
9375/9375 [==============================] - 14s 1ms/step - loss: 34.4639 - rms: 5.8706 - mae: 1.6467
Epoch 2/100
9375/9375 [==============================] - 14s 1ms/step - loss: 10.9031 - rms: 3.3020 - mae: 1.1286
Epoch 3/100
9375/9375 [==============================] - 14s 1ms/step - loss: 10.0067 - rms: 3.1633 - mae: 1.0805
Epoch 4/100
9375/9375 [==============================] - 14s 1ms/step - loss: 7.6293 - rms: 2.7621 - mae: 1.0225
Epoch 5/100
9375/9375 [==============================] - 14s 1ms/step - loss: 6.3264 - rms: 2.5152 - mae: 0.9797
Epoch 6/100
9375/9375 [==============================] - 14s 1ms/step - loss: 6.4689 - rms: 2.5434 - mae: 0.9678
Epoch 7/100
9375/9375 [==============================] - 14s 1ms/step - loss: 4.9896 - rms: 2.2337 - mae: 0.9403
Epoch 8/100
9375/9375 [==============================] - 14s 1ms/step - loss: 5.3835 - rms: 2.3202 - mae: 0.9338
Epoch 9/100
9375/9375 [==============================] - 14s 1ms/step - loss: 4.5863 - rms: 2

In [40]:
from datetime import datetime
print(datetime.now())

2021-11-14 01:45:27.297385


In [41]:
dae.save("./models/dae.keras")

In [1]:
train_dae = dae.predict(X_train)
train_dae = pd.DataFrame(data=train_dae, index=X_train.index, columns=X_train.columns)
train_dae["target"]=train["target"]

NameError: name 'dae' is not defined

In [43]:
train_dae.to_pickle("./models/train.dae.pd")

In [44]:
test_dae = dae.predict(rawtest)
test_dae = pd.DataFrame(data=test_dae, index=rawtest.index, columns=rawtest.columns)
test_dae.to_pickle("./models/test.dae.pd")